In [1]:
## Importamos todas las librerías que vamos a utilizar
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score,confusion_matrix,ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.over_sampling import RandomOverSampler 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/heart-disease/heart_2020_cleaned.csv
/kaggle/input/datosreales/DatosReales.csv
/kaggle/input/datasetheartdisease/dataset.csv


# Visualización inicial de los datos
Se realiza una visualización inicial de los datos, observando que todos los datos están transfromados a datos numéricos. También eliminamos una columna que se nos creó cuando transformamos el dataset.

In [2]:
data=pd.read_csv("/kaggle/input/datasetheartdisease/dataset.csv")
df=data.copy()
df=df.drop(["Unnamed: 0"],axis=1)
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,0,16.60,1,0,0,3.0,30.0,0,8,17,1,1,5,5.0,1,0,1
1,0,20.34,0,0,1,0.0,0.0,0,8,22,0,1,5,7.0,0,0,0
2,0,26.58,1,0,0,20.0,30.0,0,9,19,1,1,3,8.0,1,0,0
3,0,24.21,0,0,0,0.0,0.0,0,8,21,0,0,4,6.0,0,0,1
4,0,23.71,0,0,0,28.0,0.0,1,8,14,0,1,5,8.0,0,0,0


# Separamos los datos
Separamos los datos de nuestro dataset, ponemos las características que vamos a utilizar para entrenar en la variable X y la variable a predecir en la varibale Y.

In [3]:
X=df.iloc[:,1:]
Y=df.iloc[:,0]

# Estandarización de los datos
Usamos StandardScaler para estandarizar las caracteríticas que vamos a usar para entrenar en nuestro modelo.

In [4]:
scaler=StandardScaler()
columns=X.columns
s=scaler.fit_transform(X)

scaled_data=pd.DataFrame(s,columns=columns)
scaled_data.head(5)
scaled_data.shape

(319795, 16)

# Validación cruzada
Utilizamos StratifiedKFold para hacer la validación cruzada ya que tenemos un dataset desbalanceado. Se utilizarán 5 folds. Se entrenarán los siguientes modelos LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysisy GaussianNB con el fin de encontrar cual de los tres tiene mejor rendimiento.

In [5]:
modelos=[LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis,GaussianNB]
folds=5
skf=StratifiedKFold(n_splits=folds)
resultados=pd.DataFrame()
idx=0
resultados = {
    'modelo': [],
    'F1 Socre (Mean)': [],
    'F1 Socre (IC)': [],
    'Sensibilidad-Recall (Mean)': [],
    'Sensibilidad-Recall (IC)': [],
    'Accuracy (Mean)':[],
    'Accuracy (IC)':[]
}
for model in modelos:
    recall_temp = []
    accuracy_temp = []
    f1score_temp=[]
    for train, test in skf.split(X=scaled_data,y=Y):
        Xtrain, Xtest = scaled_data.iloc[train], scaled_data.iloc[test]
        Ytrain, Ytest = Y[train], Y[test]
        
        model_train=model()
        model_train.fit(Xtrain,Ytrain)
        
        Yest=model_train.predict(Xtest)
        recall_temp.append(recall_score(Ytest,Yest))
        f1score_temp.append(f1_score(y_true=Ytest,y_pred=Yest))
        accuracy_temp.append(accuracy_score(Ytest,Yest))
    resultados['modelo'].append(str(model))
    resultados['Sensibilidad-Recall (Mean)'].append(np.mean(recall_temp))
    resultados['Sensibilidad-Recall (IC)'].append(np.std(recall_temp))
    resultados['F1 Socre (Mean)'].append(np.mean(f1score_temp))
    resultados['F1 Socre (IC)'].append(np.std(f1score_temp))
    resultados['Accuracy (Mean)'].append(np.mean(accuracy_temp))
    resultados['Accuracy (IC)'].append(np.std(accuracy_temp))  


resultados = pd.DataFrame(resultados)
resultados

    
        
        

,modelo,F1 Socre (Mean),F1 Socre (IC),Sensibilidad-Recall (Mean),Sensibilidad-Recall (IC),Accuracy (Mean),Accuracy (IC)
0,<class 'sklearn.discriminant_analysis.LinearDi...,0.292204,0.008643,0.220656,0.010920,0.908566,0.001350
1,<class 'sklearn.discriminant_analysis.Quadrati...,0.341862,0.005977,0.447595,0.019739,0.852556,0.003989
2,<class 'sklearn.naive_bayes.GaussianNB'>,0.343588,0.005875,0.479598,0.020012,0.843203,0.004313


# Entrenamos el modelo seleccionado
Como se está trabajando en un problema desbalanceado nos basamos en el F1 Score y el Recall para escoger el mejor modelo. En este caso escogimos el modelo GaussianNB y el QuadraticDiscriminantAnalysis

# Partición de los datos
Partimos los datos en train y test en una proporción 70-30. 70% de los datos para entrenar el modelo y 30% de los datos para la validación del mismo.

In [6]:
X_train,X_test,y_train,y_test=train_test_split(scaled_data,
                                               Y,
                                               test_size=0.3,
                                               random_state=50)

# Cantidad de datos que hay para una clases a predecir

In [7]:
counter=Counter(y_train)
print(counter)

Counter({0: 204624, 1: 19232})


# Función para el entrenamiento de los modelos

In [8]:
def entrenarModelo(model_train, Xtrain,Ytrain,Xtest,Ytest,Xreal,Yreal):
    model=model_train()
    model.fit(X=Xtrain,y=Ytrain)
    Ypred=model.predict(Xtest)
    ## Rendimiento del entrenamiento del modelo
    print("#####################Rendimiento después del entrenamiento#########################")
    print(f'Model: {str(model)}')
    print(f'Accuracy Score: {accuracy_score(Ytest,Ypred)}')
    print(f'Precission Score: {precision_score(Ytest,Ypred)}')
    print(f'Recall Score: {recall_score(Ytest,Ypred)}')
    print(f'F1-Score: {f1_score(Ytest,Ypred)}')
    ## Rendimiento del modelo con los datos reales
    print("######################Rendimiento con los datos reales##############################")
    y_pred=model.predict(Xreal)
    print("Accuracy:",accuracy_score(Yreal, y_pred))
    print(f'Precission Score: {precision_score(Yreal,y_pred)}')
    print(f'Recall Score: {recall_score(Yreal,y_pred)}')
    print(f'F1-Score: {f1_score(Yreal,y_pred)}')

# Función para realizar el Submuestreo de los datos

In [9]:
def balancearClases(balanceador, X_train, y_train):
    Xtrain_new, Ytrain_new=balanceador.fit_resample(X_train,y_train)
    print("Dimensiones")
    print("Xtrain_new: ", Xtrain_new.shape)
    print("Ytrain_new: ", Ytrain_new.shape)
    return Xtrain_new,Ytrain_new
    

# Cargamos la data real

In [10]:
data_real=pd.read_csv('/kaggle/input/datosreales/DatosReales.csv')
data_real.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,0,"19,7",0,0,0,0,0,0,8,10,0,0,5,6,0,0,0
1,0,"22,8",0,0,0,0,0,0,8,10,0,1,5,6,1,0,0
2,1,"28,8",1,0,1,25,28,1,9,20,1,0,3,9,0,1,0
3,0,23,0,0,0,0,0,0,9,10,0,0,6,6,0,0,0
4,1,"32,4",0,1,1,30,15,1,8,18,1,0,4,4,1,1,1


Se paramos los datos reales en variables de entrada para el modelo y la clase a la que pertenece un conjunto de valores de las variables de entrada. Las variables de entrada estarán en la variable X_real y la clase a la que pertenecen estarán en la variable Y_real.

In [11]:
X_real=data_real.iloc[:,1:]
Y_real=data_real.iloc[:,0]
X_real["BMI"] = encoder.fit_transform(X_real["BMI"])

# GaussianNB

## Entrenamiento del modelo GaussianNB con UnderSampling

In [12]:
subsample=RandomUnderSampler()
Xtrain_new, Ytrain_new=balancearClases(subsample, X_train, y_train)

Dimensiones
Xtrain_new:  (38464, 16)
Ytrain_new:  (38464,)


In [13]:
entrenarModelo(GaussianNB,Xtrain_new,Ytrain_new,X_test,y_test,X_real,Y_real)

#####################Rendimiento después del entrenamiento#########################
Model: GaussianNB()
Accuracy Score: 0.797840294353704
Precission Score: 0.23331753554502369
Recall Score: 0.6047168652499693
F1-Score: 0.33671899045860265
######################Rendimiento con los datos reales##############################
Accuracy: 0.8666666666666667
Precission Score: 0.8571428571428571
Recall Score: 0.8571428571428571
F1-Score: 0.8571428571428571


## Entrenamiento del modelo GaussianNB con OverSampling

In [14]:
subsample=RandomOverSampler()
Xtrain_new, Ytrain_new=balancearClases(subsample, X_train, y_train)

Dimensiones
Xtrain_new:  (409248, 16)
Ytrain_new:  (409248,)


In [15]:
modelo_entrenado=entrenarModelo(GaussianNB,Xtrain_new,Ytrain_new,X_test,y_test,X_real,Y_real)

#####################Rendimiento después del entrenamiento#########################
Model: GaussianNB()
Accuracy Score: 0.7967041557656428
Precission Score: 0.2327233381944771
Recall Score: 0.6076649060312
F1-Score: 0.33655350704129533
######################Rendimiento con los datos reales##############################
Accuracy: 0.8666666666666667
Precission Score: 0.8571428571428571
Recall Score: 0.8571428571428571
F1-Score: 0.8571428571428571


## Entrenamiento del modelo sin ninguna técnica de submuestreo

In [16]:
entrenarModelo(GaussianNB,X_train,y_train,X_test,y_test,X_real,Y_real)

#####################Rendimiento después del entrenamiento#########################
Model: GaussianNB()
Accuracy Score: 0.8418578471737249
Precission Score: 0.2629627132357899
Recall Score: 0.4790566269500061
F1-Score: 0.33954379244297406
######################Rendimiento con los datos reales##############################
Accuracy: 0.8666666666666667
Precission Score: 0.8571428571428571
Recall Score: 0.8571428571428571
F1-Score: 0.8571428571428571


# QuadraticDiscriminantAnalysis

## Entrenamiento del modelo QuadraticDiscriminantAnalysis con UnderSampling

In [17]:
subsample=RandomUnderSampler()
Xtrain_new, Ytrain_new=balancearClases(subsample, X_train, y_train)

Dimensiones
Xtrain_new:  (38464, 16)
Ytrain_new:  (38464,)


In [18]:
entrenarModelo(QuadraticDiscriminantAnalysis,Xtrain_new,Ytrain_new,X_test,y_test,X_real,Y_real)

#####################Rendimiento después del entrenamiento#########################
Model: QuadraticDiscriminantAnalysis()
Accuracy Score: 0.7847173724971075
Precission Score: 0.22822640198080013
Recall Score: 0.6453752610244442
F1-Score: 0.33720557088761954
######################Rendimiento con los datos reales##############################
Accuracy: 0.8
Precission Score: 0.8333333333333334
Recall Score: 0.7142857142857143
F1-Score: 0.7692307692307692


## Entrenamiento del modelo QuadraticDiscriminantAnalysis con OverSampling

In [19]:
subsample=RandomOverSampler()
Xtrain_new, Ytrain_new=balancearClases(subsample, X_train, y_train)

Dimensiones
Xtrain_new:  (409248, 16)
Ytrain_new:  (409248,)


In [20]:
entrenarModelo(GaussianNB,Xtrain_new,Ytrain_new,X_test,y_test,X_real,Y_real)

#####################Rendimiento después del entrenamiento#########################
Model: GaussianNB()
Accuracy Score: 0.7971627805167867
Precission Score: 0.23300467047223664
Recall Score: 0.6066822257707898
F1-Score: 0.33669643465812255
######################Rendimiento con los datos reales##############################
Accuracy: 0.8666666666666667
Precission Score: 0.8571428571428571
Recall Score: 0.8571428571428571
F1-Score: 0.8571428571428571


## Entrenamiento del modelo sin ninguna técnica de submuestreo

In [21]:
entrenarModelo(GaussianNB,X_train,y_train,X_test,y_test,X_real,Y_real)

#####################Rendimiento después del entrenamiento#########################
Model: GaussianNB()
Accuracy Score: 0.8418578471737249
Precission Score: 0.2629627132357899
Recall Score: 0.4790566269500061
F1-Score: 0.33954379244297406
######################Rendimiento con los datos reales##############################
Accuracy: 0.8666666666666667
Precission Score: 0.8571428571428571
Recall Score: 0.8571428571428571
F1-Score: 0.8571428571428571
